In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
import time
import requests



In [ ]:
BASE_URL = os.environ["OLLAMA_BASE_URL"]
MODEL_A = os.environ["OLLAMA_MODEL_A"]
MODEL_B = os.environ["OLLAMA_MODEL_B"]

def call_ollama(model, prompt):
    print(f"⏳ Calling {model}...", flush=True)

    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "num_predict": 80
        }
    }

    r = requests.post(
        f"{BASE_URL}/api/generate",
        json=payload,
        timeout=20
    )

    data = r.json()
    if "response" not in data:
        raise RuntimeError(data)

    return data["response"].strip()


def debate(topic, duration_seconds=30):
    start = time.time()
    history = [f"Debate topic: {topic}"]
    turn = 0

    print(f"\n🔥 DEBATE STARTED: {topic}\n")

    while time.time() - start < duration_seconds:
        context = "\n".join(history[-3:])

        if turn == 0:
            prompt = f"""
You are Agent A.
Argue FOR the topic.
You must disagree with the opponent.

Context:
{context}

Respond:
"""
            try:
                reply = call_ollama(MODEL_A, prompt)
            except Exception as e:
                reply = f"[Agent A error: {e}]"
            speaker = "Agent A (FOR)"
        else:
            prompt = f"""
You are Agent B.
Argue AGAINST the topic.
You must disagree with the opponent.

Context:
{context}

Respond:
"""
            try:
                reply = call_ollama(MODEL_B, prompt)
            except Exception as e:
                reply = f"[Agent B error: {e}]"
            speaker = "Agent B (AGAINST)"

        history.append(reply)
        print(f"{speaker}:\n{reply}\n{'-'*50}")

        turn = 1 - turn
        time.sleep(2)

    print("\n🛑 DEBATE ENDED\n")


In [ ]:
debate(
    topic="Should governments heavily regulate artificial intelligence?",
    duration_seconds=30
)
